We will start by importing all the stuff we need (be ready - the list is long...)

In [ ]:
import os
import json
import sys
import shutil
from datetime import datetime 
import time
from dateutil import parser
import pymongo
from pymongo import MongoClient
import re
import requests
from requests.auth import HTTPBasicAuth
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

from itertools import product
import rasterio as rio
from rasterio import windows
import rasterio.features
import rasterio.warp

import analona
from dateutil import parser
from datetime import datetime

Now we'd like to set up functions for accesing the mosaic API:

In [ ]:
mosaic_url = "https://api.planet.com/basemaps/v1/mosaics/"
planet_api_key = '-paste-here-'

# Init the session object
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)
session.auth = (planet_api_key, '')

And the set up the mongo client:

In [ ]:
mongo_uri = ''
client = pymongo.MongoClient(mongo_uri)
db = client.louvre
buildings_collection = db.buildings

After doing so it's finally time to get to work!

The goal is to write a function which - given a raster of building indentifications, turns each building in the raster into a geojson representation which we then add to an item we can varify and push into Louvre.
<br>The problem arise with converting the raster to a geojson - the raster as is is just too big for a computer to process and extract all the buildings in it. Because of that we split each tiff file we get into tiles of size 256x256 and then extract the items from each of the tiles.
<br>The raster buildings are either in the Green mask of an RGB raster, or in the White mask of a BW raster.

The first thing we will do then is to write a code which splits the image into subtiles, we will use *rio* package to do so:

In [ ]:
def get_tiles(ds, width=256, height=256):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window = windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform
        

def split_file(filepath):
    folder = 'tmp_for_{}'.format(filepath.split('/')[1])
    if os.path.exists(folder):
        print("already split {}".format(filepath.split('/')[1]))
        return
    
    os.mkdir(folder)
    with rio.open(filepath) as inds:
        meta = inds.meta.copy()
        for window, transform in get_tiles(inds):
            meta['transform'] = transform
            meta['width'], meta['height'] = window.width, window.height
            outpath = os.path.join(folder,'{}_{}'.format(int(window.col_off), int(window.row_off)))
            with rio.open(outpath, 'w', **meta) as outds:
                outds.write(inds.read(window=window))

Now after doing so we can write a function which takes a filepath which leads to a raster, the start and end date of the mosaic, and the download url, and construct and pushes an item to Louvre.

In [ ]:
def turn_to_json(filepath, mosaic_start_date, mosaic_end_date, url):
    folder = 'tmp_for_{}'.format(filepath.split('/')[1])
    
    #The first thing we do is split the original file
    split_file(filepath)
    
    #Now the folder should contain all splited tiles.
    for _, _, files in os.walk(folder):  
        for filename in files:
            file_route = os.path.join(folder, filename)
            with rio.open(file_route) as dataset:
                _, _, image_id = filepath.split('/')
                image_id = image_id.split('.')[0]
                
                # Init a counter for different IDs
                counter = 0
                # RGB Raster
                if(dataset.count == 4):
                    # Read the dataset's valid data mask as a ndarray.
                    mask = dataset.read(3)
                # BW Raster
                elif(dataset.count == 2):
                    mask = dataset.read(1)
                    
                shp = rio.features.shapes(
                        mask, transform=dataset.transform)
                # Extract feature shapes and values from the array.
                for geom, _ in shp:
                    item = {}
                    
                    # Transform shapes from the dataset's own coordinate
                    # reference system to CRS84 (EPSG:4326).
                    geom = rio.warp.transform_geom(
                        dataset.crs, 'EPSG:4326', geom, precision=6)

                    item['_id'] = 'planet_building_{}_{}_{}'.format(image_id, filename, counter)
                    item['geometry'] = geom
                    item['company'] = 'Planet'
                    item['analyticsDeliveryTime'] = {
                        'start': datetime.utcfromtimestamp(int(round(mosaic_start_date.timestamp()))),
                        'end': datetime.utcfromtimestamp(int(round(mosaic_end_date.timestamp())))
                    }
                    item['analyticsInfo'] = {
                        'url': url,
                        'storage': 'Planet'
                    }
                    item['sourceImagesIds'] = [image_id]
                    is_valid = analona.Building(item).validate()
                    counter += 1
                    if(is_valid == True):
                        buildings_collection.update({ '_id': item['_id'] }, item, upsert = True)
                        print('pushed to db:', item['_id'])
                    else:
                        print('did not pass varification- {}'.format(is_valid))
    #Remove the folder of tiles
    shutil.rmtree(folder)
    #Remove the image because we don't need it anymore
    os.remove(filepath)

Now what we need to do is to modify the download code (originally from 'https://github.com/nsplt/planet_downloader/blob/master/basemap.py'):

In [ ]:
def download_quad(mosaic_name, mosaic_start_date, mosaic_end_date, quad):
    directory = ''
    url = quad['_links']['download']
    if(url):
        directory = '{}/{}'.format(str(download_directory),\
                                    str(mosaic_name))

        if not os.path.exists(directory):
            os.makedirs(directory)

        res = session.get(url, stream = True)

        if res.status_code != 200:
            raise Exception('Error while getting quads: ' + res.text)

        filename = str(re.findall('filename=(.+)', str(res.headers.get('Content-Disposition')))[0])

        filepath = '{}/{}'.format(str(directory), filename)

        print('downloading to:', filepath)
        with open(filepath, 'wb') as f:
            for chunk in res.iter_content(chunk_size = 1024 * 1024):
                if chunk:
                    f.write(chunk)

        #After we download the file locally, we can transform it and insert to louvre
        turn_to_json(filepath, mosaic_start_date, mosaic_end_date, url)        
    else:
        print('no available download')

In [ ]:
def handle_quads_batch(mosaic_name, mosaic_start_date, mosaic_end_date, quad):
    quad_items = quad['items']
    for item in quad_items:        
        download_quad(mosaic_name, mosaic_start_date, mosaic_end_date, item)

In [ ]:
def fetch_quads(mosaic_name, mosaic_start_date, mosaic_end_date, quad_items):
    handle_quads_batch(mosaic_name, mosaic_start_date, mosaic_end_date, quad_items)
    next_url = quad_items['_links'].get('_next')

    if next_url:
        next = session.get(next_url).json()
        fetch_quads(mosaic_name, mosaic_start_date, mosaic_end_date, next)

In [ ]:
def is_overlap(start1, start2, end1, end2):
    return max(start1, start2) <= min(end1, end2)

def overlap_bbox(lx1, lx2, ly1, ly2, ux1, ux2, uy1, uy2):
    x_left = max(lx1, lx2)
    y_bottom = max(ly1, ly2)
    x_right = min(ux1, ux2)
    y_top = min(uy1, uy2)

    if(x_left > x_right or y_bottom > y_top):
        return False
    else:
        bbox = [x_left, y_bottom, x_right, y_top]
        return bbox

In [ ]:
def download_mosaic(mosaic):
    mosaic_id = mosaic['id']
    print('Mosaic Name', mosaic['name'])
    mosaic_start_date = datetime.strptime(mosaic['first_acquired'].split('T')[0], '%Y-%m-%d')
    mosaic_end_date =  datetime.strptime(mosaic['last_acquired'].split('T')[0], '%Y-%m-%d')

    if(is_overlap(mosaic_start_date, start_date, mosaic_end_date, end_date)):
        mosaic_bbox = mosaic['bbox']

        overlap = overlap_bbox(mosaic_bbox[0], bbox[0], mosaic_bbox[1], bbox[1], mosaic_bbox[2], bbox[2], mosaic_bbox[3], bbox[3])

        if(overlap is False):
            print('There is no overlap between the mosaic and the bbox.')
        else:
            bbox_str = str(overlap[0]) + ',' + str(overlap[1]) + ',' + str(overlap[2]) + ',' + str(overlap[3])
            quads = mosaic_url + mosaic_id + '/quads?bbox=' + bbox_str  
            quads_res = session.get(quads).json()
            fetch_quads(mosaic['name'], mosaic_start_date, mosaic_end_date, quads_res)
    else:
        print('There is no overlap between the dates.')

And now we can run the code with a bounding box and dates:

In [ ]:
download_directory = '.'
basemap_types = ['buildings']
start_date = datetime.strptime("2016-09-01T06:11:00.000Z".split('T')[0], "%Y-%m-%d") 
end_date = datetime.strptime("2018-11-01T06:11:00.000Z".split('T')[0], "%Y-%m-%d") 
bbox = [35.89233398437499, 32.68099643258195, 36.37573242187499, 33.169743600216165]

mosaics = session.get(mosaic_url).json()
mosaics_list = mosaics['mosaics']

for mosaic in mosaics_list:
    mosaic_name = mosaic['name']
    should_download = any(mosaic_name.find(type) != -1 for type in basemap_types)

    if should_download:
        download_mosaic(mosaic)